In [6]:
%load_ext autoreload
%autoreload 2
import datetime, urllib, re, math, json, requests,io
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
from scipy import optimize
import scipy.signal
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML
from ipywidgets import *


try:
    from packaging import version
except ImportError:
    !pip install packaging

if( version.parse(pd.__version__) < version.parse("0.23.4")):
    print("update pandas")
    !pip install pandas --upgrade #--ignore-installed
    print(version.parse(pd.__version__))
    
import compute    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#get the data
df = pd.read_csv("https://coronadatascraper.com/timeseries-tidy.csv")
df.fillna("-1",inplace=True);
dataTCS =df.set_index(["city","county","state","country","type"])
dataTCS["date"] = [datetime.datetime.strptime(ts,"%Y-%m-%d").date() for ts in dataTCS["date"]]


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#set compute dataframe
compute.dataTCS = dataTCS

In [9]:
from ipywidgets import *


interact(compute.plotter,minCount=(0,1e6,1e4),
         minCountKpi=dataTCS.index.levels[4].unique().values,
#         equalize=[True, False],
         equalizeTrg=dataTCS.index.levels[3].unique().values,
         equalizeCount=(0,10000),
         plotKpi=dataTCS.index.levels[4].unique().values
#         logyPlot=[True, False],        
#         reference=[True,False]
         ,xlim=(-1,60)
         ,ylim=(-1,1e6)
        );#countryStateSweep=dataTCS.index.levels[3].unique().values)

interactive(children=(FloatSlider(value=5000.0, description='minCount', max=1000000.0, step=10000.0), Dropdown…

In [10]:
# states
countryStateSweep="USA"
idx = pd.IndexSlice

interact(compute.plotter,minCount=(0,1e5),
         minCountKpi=dataTCS.index.levels[4].unique().values,
         #equalize=[True, False],
         equalizeTrg=np.sort(np.unique(dataTCS.loc[idx["-1","-1",:,countryStateSweep,"cases"]].index.values))[1:],
         equalizeCount=(0,10000),
         plotKpi=dataTCS.index.levels[4].unique().values,
         #logyPlot=[True, False],
         countryStateSweep=countryStateSweep
        );


interactive(children=(FloatSlider(value=5000.0, description='minCount', max=100000.0), Dropdown(description='m…

In [11]:
compute.table_gen(dataTCS)



C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\colors.py:512: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [28]:
#country
country_w = widgets.Dropdown(
    options=dataTCS.index.levels[3],
    value='USA',
    description='Country:',
    disabled=False,
)

#state
#states=()
def getStates(ctr):
    idx    = pd.IndexSlice['-1','-1',:,ctr,"cases"]
    g = dataTCS.loc[idx,:][dataTCS.loc[idx,"date"] == datetime.date(2020,4,1) ]
    return np.sort([c[2] for c  in g.index.values])
def updateStates_e(*args):
    state_w.options = updateStates(country_w.value)
    
    print(state_w.options)
print(getStates("USA"))
state_w = widgets.Dropdown(
    options    = getStates("USA"),#updateStates(country_w.value),
    value      = 'CA',
    description='State:',
    disabled   = False,
)
country_w.observe(updateStates_e, 'value')
def getCounties(country, state):
    idx    = pd.IndexSlice['-1',:,state,country,"cases"]
    g      = dataTCS.loc[idx,:][dataTCS.loc[idx,"date"] == datetime.date(2020,4,1) ]
    return np.sort([c[1] for c  in g.index.values])
def updateCounty_e(*args):
    state_w.options = getCounties(country_w.value, county_w.value)

county_w = widgets.Dropdown(
    options    = getCounties("USA","CA"),
    value      = '-1',
    description='County:',
    disabled   = False,
)
state_w.observe(updateCounty_e, 'value')



interact(projector,
        stype='cases',
    city   ='-1',
    county ='-1',
    state ="CA",
    country="USA",
    minCount = (0,1e5,1e2)
        )

['-1' 'AK' 'AL' 'AR' 'AS' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'GU'
 'HI' 'IA' 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO'
 'MP' 'MS' 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR'
 'PA' 'PR' 'RI' 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VI' 'VT' 'WA' 'WI' 'WV'
 'WY']


Dropdown(description='Country:', index=180, options=('AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ATG', '…

Dropdown(description='State:', index=6, options=('-1', 'AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', '…

Dropdown(description='County:', options=('-1', 'Alameda County', 'Alpine County', 'Amador County', 'Butte Coun…

In [22]:
    idx    = pd.IndexSlice['-1','-1',:,"USA","cases"]
    g =dataTCS.loc[idx,:][dataTCS.loc[idx,"date"] == datetime.date(2020,4,1) ]
    np.sort([c[2] for c  in g.index.values])

array(['-1', 'AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE',
       'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA',
       'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND',
       'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR',
       'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI',
       'WV', 'WY'], dtype='<U2')